### Create some context text

We're using two simple sentences as an example. But you could grab any docs you need, loop through them, and store your text for retrieval. 

In [31]:
sports_text = "The Giants and Dodgers play today, March 17, at 5pm."
finance_text = "The price of WDY stock was $272.45 on March 17."

### Call the embedding

#### Create a bedrock runtime

In [32]:
import boto3
bedrock_runtime = boto3.client(region_name = 'us-east-1', service_name='bedrock-runtime')

#### Create a JSON request for the embedding

Here we're passing the sports_text into the response body so we can check out its embeddings.

In [33]:
import json
json_request = {'inputText': sports_text}
body = json.dumps(json_request)
response = bedrock_runtime.invoke_model(body=body, modelId = 'amazon.titan-embed-text-v1')

In [34]:
response_body = json.loads(response.get('body').read())

We see see the embedding dimension size using the len() function. 

In [35]:
len(response_body['embedding'])

1536

#### Functionalize the request for retrieving the embedding

In [36]:
def embed_text(text):
    json_request = {'inputText': text}
    body = json.dumps(json_request)
    response = bedrock_runtime.invoke_model(body=body, modelId = 'amazon.titan-embed-text-v1')
    return json.loads(response.get('body').read())['embedding']

In [37]:
embed_text("How are you?")

[0.083496094,
 -0.83203125,
 -0.46875,
 -0.18652344,
 0.765625,
 0.4140625,
 0.3125,
 -0.0008239746,
 -0.29882812,
 0.47070312,
 0.8671875,
 -0.3046875,
 -0.15917969,
 0.04711914,
 -0.48242188,
 -0.052246094,
 -0.45898438,
 -0.26953125,
 -0.6953125,
 0.1484375,
 0.088378906,
 0.0859375,
 0.08544922,
 -0.21777344,
 0.14453125,
 0.53515625,
 0.73828125,
 -0.22265625,
 0.34960938,
 0.06542969,
 0.11376953,
 0.0014648438,
 0.17089844,
 -0.625,
 0.22167969,
 -0.47460938,
 0.90625,
 -0.42773438,
 0.265625,
 -0.47460938,
 0.70703125,
 -0.42578125,
 0.020996094,
 0.734375,
 -0.19433594,
 -0.055664062,
 0.061767578,
 -0.51171875,
 0.36328125,
 1.125,
 0.51953125,
 -0.5,
 0.042236328,
 0.24902344,
 0.32421875,
 -0.03930664,
 -0.63671875,
 -0.15039062,
 -0.016723633,
 0.08886719,
 0.359375,
 0.09277344,
 -0.24023438,
 -0.65234375,
 -0.625,
 -0.18359375,
 -0.5546875,
 -0.34960938,
 0.52734375,
 -0.19335938,
 -0.43945312,
 0.24804688,
 0.4140625,
 0.055664062,
 -0.453125,
 -0.27734375,
 0.41796875,

### Organize the embeddings

In [6]:
import pandas as pd
data = {'name': ['sports_text', 'finance_text'], 'text': [sports_text, finance_text]}
df = pd.DataFrame(data)

In [7]:
df

,name,text
0,sports_text,"The Giants and Dodgers play today, March 17, a..."
1,finance_text,The price of WDY stock was $272.45 on March 17.


We use the .apply method to transform our text and see their embeddings.

In [8]:
df['embedding'] = df['text'].apply(embed_text)
df

,name,text,embedding
0,sports_text,"The Giants and Dodgers play today, March 17, a...","[0.8515625, 0.421875, -0.48828125, 0.010253906..."
1,finance_text,The price of WDY stock was $272.45 on March 17.,"[0.19433594, -0.13378906, -0.37890625, 0.59765..."


In this example I'm using a Pandas dataframe as the storage structure for the text and embeddings. In a real-world application, this might be done in an actual vector database.

### Identify the cosine similarity

Cosine similarity is the dot product of two vectors divided by the magnitude of the vectors multiplied by each other. Cosine similarity is used to identify how similar two vectors are to each other.

In [9]:
import numpy as np
def cosine_similarity(v1, v2):
    vector1 = np.array(v1)
    vector2 = np.array(v2)
    
    dot_product = np.dot(np.array(v1), np.array(v2))
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)

    return dot_product / (magnitude_v1 * magnitude_v2)

In [10]:
vec1 = df['embedding'][0] 

In [11]:
vec2 = df['embedding'][1]

In [12]:
cosine_similarity(vec1, vec2) 

0.0250861808710861

### Retrieve the most similar documents to a query

In [13]:
query = "What was the price of WDY stock on March 17?"

In [14]:
# embed the prompt

prompt_embedding = embed_text(query) 

In [15]:
# calculate the similarity of the prompt to each text

df['prompt_similarity'] = df['embedding'].apply(lambda vector: cosine_similarity(vector, prompt_embedding))

In [16]:
df

,name,text,embedding,prompt_similarity
0,sports_text,"The Giants and Dodgers play today, March 17, a...","[0.8515625, 0.421875, -0.48828125, 0.010253906...",0.126192
1,finance_text,The price of WDY stock was $272.45 on March 17.,"[0.19433594, -0.13378906, -0.37890625, 0.59765...",0.792362


In [17]:
 # Get the text with the highest similarity

most_similar_text = df.nlargest(1, 'prompt_similarity').iloc[0]['text']


### Create the prompt and inject the most_similar_text

In [18]:
full_prompt = f"Answer this question based on the context provided. Here is the question: \n{query} \n\nHere is the context: \n{most_similar_text} Provid a simple answer to the question."

In [19]:
print(full_prompt)

Answer this question based on the context provided. Here is the question: 
What was the price of WDY stock on March 17? 

Here is the context: 
The price of WDY stock was $272.45 on March 17. Provid a simple answer to the question.


### Retrieve the answer

In [20]:
body = json.dumps({'prompt': full_prompt, 'max_gen_len': 1024, 'temperature': 0.9, 'top_p': 0.9})

In [24]:
response = bedrock_runtime.invoke_model(body=body, modelId='meta.llama2-70b-chat-v1')

In [25]:
response_body = json.loads(response.get('body').read())

In [26]:
response_body['generation']

'\n\nAnswer: \n$272.45'